# ANALYSIS OF MTOR

This notebook contains an analysis of the mTOR system described in Varusai and Nguyen, 2018.

We consider a model of the signaling network for mTOR, the mechanistic Target of Rapamycin (mTOR), an evolutionarily conserved network that controls key cellular processes, especially cell growth and metabolism. The signalling cascade is activated by a variety of receptor tyrosine kinases, most notably the Insulin Receptor.

<img src="images/mTOR-System.png"
     alt="Markdown Monster icon" 
     width="500" height="600"
     style="float: left; margin-right: 10px;" />

In [1]:
from controlSBML import ControlSBML

import control
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'controlSBML'

# Problem Statement

The biological problem we address is controlling the concentration of phosphorylated mTORC1 through phosphorylated insulin. \ccite{varusai_dynamic_2018} describes this in terms of the species pIRS. The system we study has the single input $pIRS$ and the output mTORC1. It turns out that the mTOR system is bistable with respect to the insulin. Thus, our control objective is to regulate the outputs in the high insulin region where V1 $> 4$.
* Outputs: pmTORC1, pmTORC2
* Inputs: V1
* Control problem: regulate pmTORC1, pmTORC2

## mTOR Model

In [ ]:
URL = "https://www.ebi.ac.uk/biomodels/services/download/get-files/MODEL1909250003/2/Varusai2018.xml"
CTLSB = ControlSBML(URL, figsize=(5, 5), times=np.linspace(0, 2000, 20000), markers=False,
                   save_path="data.csv")  # Specify default value of options

In [ ]:
ts = CTLSB.plotModel(ax2=0)

In [ ]:
ts.head()

## System Definition

In [ ]:
INPUT_NAME = "pIRS"
OUTPUT_NAME = "pmTORC1"

In [ ]:

df = ts[[INPUT_NAME, OUTPUT_NAME]]
seconds = np.array(df.index)/1000
df.index = seconds
df.plot()

In [ ]:
# Define the system and plot response over a controlled range
CTLSB.setSystem(input_name=INPUT_NAME, output_name=OUTPUT_NAME)
_ = CTLSB.plotStaircaseResponse(initial_value=0, final_value=35, times=np.linspace(0, 10000, 100000))

Clearly, the effect of theinput, pIRS, is effective over only a subset of the range of pIRS. So, we focus on the interval [20, 25].

In [ ]:
_, builder = CTLSB.plotStaircaseResponse(initial_value=20, final_value=25, times=np.linspace(0, 10000, 100000))

In [ ]:
builder

* The system is controllable in the region V1 $>4$, although the control decreases with larger V1 for both complexes.
* The operating region for V1 implies an operating region for pmTORC1 of 180 to 220 and for pmTORC2 0.7 to 1.0.

# System Identification

In [ ]:
_, builder = CTLSB.plotTransferFunctionFit(num_numerator=2, num_denominator=3, initial_value=20, final_value=25,
                                  fit_start_time=2000, times=np.linspace(0, 10000, 100000))

Note that the sole pole is at 0. This is consistent with an oscillatory system.

In [ ]:
builder

In [ ]:
TRANSFER_FUNCTION = CTLSB.getOpenLoopTransferFunction()
TRANSFER_FUNCTION

In [ ]:
TRANSFER_FUNCTION.poles(), TRANSFER_FUNCTION.zeros()

In [ ]:
_ = control.bode(TRANSFER_FUNCTION)

# Testbed Construction

Here, we build a closed loop system where the open loop is the SBML model of the simulated system. Our has a PI controller.

In [ ]:
_, builder = CTLSB.plotClosedLoop(setpoint=150, kp=1, ki=0.001, kf=None)

In [ ]:
builder

In [ ]:
_ = CTLSB.plotClosedLoop(setpoint=150, kp=1, ki=0.001, kf=None, times=np.linspace(0, 2000, 20000))

In [ ]:
_ = CTLSB.plotClosedLoop(setpoint=80, kp=1, ki=0.01, kf=None)

We are able to simulate the closed loop system, but it's not easy to find design parameters such that: (a) settling times are short; (b) there is not overshoot; and (c) all concentrations are non-negative.

**Notes**
1. Need to use positive feedback. Not clear from the controlability curves.
2. Introducing even a little I behavior makes the controller worse.
3. There is a multimodal behavior of the network that radically changes the long term steady state values. Could this be analyzed by decomposing the system to have multiple tandem controllers. The first controls the effect of IR on pIRS; the second controls the effect of pIRS on pmTORC1.

# Design and Evaluation

In [ ]:
grid = CTLSB.getGrid(kp_spec=True, ki_spec=False, num_coordinate=100, is_random=False)
axis = grid.getAxis("kp")
axis.setMinValue(0.1)
axis.setMaxValue(10)
ts, builder = CTLSB.plotGridDesign(grid, setpoint=120, num_restart=1, is_greedy=False, save_path="data1.csv")

In [ ]:
CTLSB.plotDesignResult(figsize=(5,5), title="One Parameter Designs", save_path="data1.csv")

In [ ]:
grid = CTLSB.getGrid(kp_spec=True, ki_spec=True, num_coordinate=11, is_random=False)
axis = grid.getAxis("kp")
axis.setMinValue(0.1)
axis.setMaxValue(2)
axis = grid.getAxis("ki")
axis.setMinValue(0.01)
axis.setMaxValue(0.02)
ts, builder = CTLSB.plotGridDesign(grid, setpoint=120, num_restart=1, is_greedy=False, is_report=True, save_path="data2.csv")

In [ ]:
CTLSB.plotDesignResult(figsize=(5,5), save_path="data2.csv", title="Two Parameter Design")

In [ ]:
grid = CTLSB.getGrid(kp_spec=True, ki_spec=True, kf_spec=True, num_coordinate=11, is_random=False)
axis = grid.getAxis("kp")
axis.setMinValue(1.5)
axis.setMaxValue(1.9)
axis.setNumCoordinate(11)
axis = grid.getAxis("ki")
axis.setMinValue(0.001)
axis.setMaxValue(0.009)
axis.setNumCoordinate(11)
axis = grid.getAxis("kf")
axis.setMinValue(0.001)
axis.setMaxValue(0.002)
axis.setNumCoordinate(11)
ts, builder = CTLSB.plotGridDesign(grid, setpoint=120, num_restart=1, is_greedy=False, is_report=True, save_path="data3.csv")

In [ ]:
CTLSB.plotDesignResult(figsize=(5,5), save_path="data3.csv", suptitle="Three Parameter Design")

Move x-axis label to far left

In [ ]:
_ = CTLSB.plotClosedLoop(setpoint=120, kp=CTLSB.kp, ki=CTLSB.ki, times=np.linspace(0, 20000, 200000))

In [ ]:
_ = CTLSB.plotClosedLoop(setpoint=150, kp=2.2084, ki=0.0188, times=np.linspace(0, 2000, 20000))

# Issues

1. Handle longer time strings on x-axis